In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

由于我们在上一讲已经做过数据预览,所以这里不进行数据可视化和清洗等工作. 

数据是全部有标注的,我们将标注全部转换为数字后仅保留20%的标注数据,将其余数据的标注变为空


In [2]:
df = pd.read_csv('./data/cleaned_data.csv')

df.describe()

,Age,Work_Experience,Spending_Score,Family_Size,Male,Yes,Yes.1,Doctor,Engineer,Entertainment,...,Healthcare,Homemaker,Lawyer,Marketing,Cat_2,Cat_3,Cat_4,Cat_5,Cat_6,Cat_7
count,10695.000000,10695.000000,10695.000000,10695.000000,10695.000000,10695.000000,10695.000000,10695.000000,10695.000000,10695.000000,...,10695.000000,10695.000000,10695.000000,10695.000000,10695.000000,10695.000000,10695.000000,10695.000000,10695.000000,10695.000000
mean,0.359322,0.175249,0.271295,0.231323,0.546143,0.594016,0.623843,0.086957,0.087424,0.116877,...,0.163628,0.030669,0.078915,0.037681,0.052641,0.101823,0.137915,0.010659,0.656194,0.025152
std,0.236256,0.232101,0.370073,0.188028,0.497890,0.491104,0.484443,0.281785,0.282469,0.321289,...,0.369955,0.172426,0.269619,0.190433,0.223327,0.302430,0.344827,0.102696,0.474999,0.156594
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.169014,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.323944,0.071429,0.000000,0.250000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,0.492958,0.214286,0.500000,0.375000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
def convert_label(x):
    # 这里偷懒使用ASCII码.这里的意思是把A, B, C, D转为0, 1, 2, 3
    return ord(x) - 65

labels = df['Segmentation'].apply(lambda x: convert_label(x));

x = df.drop('Segmentation', axis = 1)
np.random.seed(1)
rd = np.random.rand(len(df))

# 我们选取大概20%的数据作为有标记数据,其余为无标记数据, 这里会把random值小于0.8的行变成true(需要变成无标记数据)
rd = rd < 0.8

# 把真实标签存下来,方便最后检查结果
real_y = labels[rd]

# 变为-1,即无标记数据
labels[rd] = -1

In [4]:
labeled = labels[~rd]
nolabel = labels[rd]
print("无标记数据共{}个".format(len(nolabel)))
print("有标记数据共{}个".format(len(labeled)))

无标记数据共8598个
有标记数据共2097个


查看一下无标记数据数量,以及各个标记现在在改造后数据集的状况

In [5]:
labels.value_counts()

-1    8598
 3     604
 0     544
 2     490
 1     459
Name: Segmentation, dtype: int64

In [6]:
from sklearn.semi_supervised import LabelPropagation, LabelSpreading

In [7]:
lp =LabelSpreading()
lp.fit(x, labels)
y_pred = lp.predict(x)

# 重新预测-1部分
y_pred = y_pred[rd]

In [8]:
from sklearn.metrics import accuracy_score,recall_score,f1_score

In [9]:
print('准确率{}%'.format(accuracy_score(real_y,y_pred) * 100))

准确率40.5908350779251%


In [10]:
acc = accuracy_score(real_y,y_pred)
compared_acc = (acc * len(nolabel) + 1 * len(labeled))/len(df)
print("当和无监督学习方法对比时, 半监督学习的准确率为{}%".format(compared_acc * 100))

当和无监督学习方法对比时, 半监督学习的准确率为52.2393641888733%
